In [2]:
from numpy import nan, zeros
import re
import numpy as np
import pandas as pd
from copy import deepcopy
import seaborn as sns
from copy import deepcopy
from pandas import read_csv, get_dummies, Series
from sklearn.preprocessing import PowerTransformer
import warnings
warnings.filterwarnings("ignore")

In [41]:
class execute_data:
    """Input Class"""
    
    def __init__(self,filename,target,check_only_flag=False,missing_headers=False):
        self.filename=filename
        self.check_only_flag=check_only_flag
        self.missing_headers=missing_headers
        self.target=target
        
    def load_csv(self):
        """Read data as csv and return as pandas data frame."""
        
        filePath=self.filename
        missing_headers=self.missing_headers
        print("Reading Data From : "+ filePath.upper())
        if missing_headers:
            data = read_csv(filePath, header=None)
        else:
            data = read_csv(filePath, header=0)
            
        global rows, cols
        rows, cols = data.shape
        data=self.whitespace_removal(data)
        
        return data
    
    def load_csv_test(self,**kwargs):
        """Read data as csv and return as pandas data frame."""
        
        options = {
            'column_exception_list' : [''],
            'categorical_columns_list' : [''],
            'currency_columns' : [''],
            'currency_symbols_list':[''],
            'numeric_columns_list':[''],
        }
        options.update(kwargs)
        print(options)
        filePath=self.filename
        missing_headers=self.missing_headers
        print("Reading Data From : "+ filePath.upper())
        if missing_headers:
            data = read_csv(filePath, header=None)
        else:
            data = read_csv(filePath, header=0)
    
        # make shape of data frame global
        global rows, cols
        rows, cols = data.shape
        data=self.whitespace_removal(data)
        
        return data
     
    def whitespace_removal(self,data):
        """Removes Whitespace from data"""
        
        print("Removing Whitespace From Data")
        data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
        data=self.remove_redundant_columns(data)
        
        return data
    
    def remove_redundant_columns(self,data):
        """Remove Redundant Columns Empty Columns and Grain Columns"""
        
        print("Removing Redundant Columns From Data")
        redundant_list=[]
        d_series=data.isna().sum()
        frame=pd.DataFrame([d_series])
        frame_unique=pd.DataFrame([data.nunique()])
        for i in frame_unique.columns:
            if(frame_unique[i][0]>(0.9)*data.shape[0]):
                data.drop([i],axis=1,inplace=True)
                redundant_list.append(i)
        for i in frame.columns:
            if(frame[i][0]>data.shape[0]/10 and data.shape[1]>10 and data.shape[0]>10000):
                data.drop([i],axis=1,inplace=True)
                redundant_list.append(i)
        print("Columns Removed are:"+str(redundant_list))
        data=self.drop_duplicated_data(data)
        
        return data
    
    def drop_duplicated_data(self,data):
        """Removed duplicates from data"""
        
        print("Dropping Duplicated Data")
        print("# of Duplicate Data Records : "+str(data[data.duplicated()==True].shape[0]))
        data.drop_duplicates(inplace=True)
        self.list_numeric=self.extract_numeric_columns(data)
        data=self.replace_missing_data(data)
        
        return data
    
    def extract_numeric_columns(self,data):
        """Extracting Numeric Columns from string type.
        Also, generating list of all numeric type columns detected."""
        
        print("Extracting Numeric Columns")
        numeric_cols=[]
        numeric_string_cols=[]
        col_list=list(data.select_dtypes(exclude=[np.number]).columns)
        for col in col_list:
            val=data[col][data[col].astype(str).str.contains("^\d+,{0,1}\d+$",na = False)]
            if(val.all() and val.size>rows/100):
                numeric_cols.append(col)
                numeric_string_cols.append(col)
        print("List of Numeric columns : " + str(numeric_cols))
        for col in numeric_string_cols:
            data[col] = data[col].astype(str).str.replace(",","").astype(float)
        numeric_cols.extend(list(data.select_dtypes(exclude=['object']).columns))
        numeric_cols=list(set(numeric_cols))
        
        return numeric_cols
    
    def replace_missing_data(self,data):
        """Replace missing data values and return as pandas data frame."""
        
        print("Replace Missing Data")
        data = data.replace('?', nan)

        nan_vals = dict(data.count(axis=1))
        nan_vals = {key: value for (key, value) in nan_vals.items() if value < data.shape[1]-2}
        print("Rows affected are : "+ str(len(nan_vals)))
        data = data.drop(index=nan_vals.keys())
        data=self.categorical_variables(data)
        return data
    
    def categorical_variables(self,data):
        """Identify Categorical Variables and perform 1-Hot Encoding for them"""
        
        print("Identifying categorical variables")
        categorical_variables=[]
        text_columns=list(data.select_dtypes(exclude=[np.number]).columns)
        frame_unique=pd.DataFrame([data[text_columns].nunique()])
        for i in frame_unique.columns:
            if(frame_unique[i][0]<(0.005)*data.shape[0]):
                categorical_variables.append(i)
        print(categorical_variables)
        if(len(categorical_variables)>0):
            data=self.one_hot_encoding(data,categorical_variables)
        self.outlier_list(data,self.list_numeric)
        data=self.remove_outliers(data,self.list_numeric)
        return data
    
    def one_hot_encoding(self,data,categorical_variables):
        """Perform a one-hot encoding and return as pandas data frame."""
        
        df=data[categorical_variables]
        for i in categorical_variables:
            data.drop(i,axis=1)
        df=get_dummies(df)
        
        return data.join(df)

    def outlier_list(self,data,list_numeric_cols):
        """Produces a list of numeric columns that contain any outlier 
        and the count of outliers for the same."""
        
        print("Checking Dataset For Outliers")
        outlier_list_count=[]
        mean = data.describe().iloc[1, :]
        std = data.describe().iloc[2, :]
        for (col, mean, std) in zip(list_numeric_cols, mean, std):
            count=0
            for i in data[col]:
                if(i> 3*std + mean):
                    count+=1
            outlier_list_count.append([col,count])
        print(outlier_list_count)
        
    def remove_outliers(self,data, list_numeric_cols):
        """Remove outliers from data and return as a pandas data frame."""
        
        print("Remove Outliers for Dataset")
        mean = data.describe().iloc[1, :]
        std = data.describe().iloc[2, :]
        
        for (list_numeric_cols, mean, std) in zip(list_numeric_cols, mean, std):
            data = data[abs(data[list_numeric_cols]) < 3*std + mean]
        data=self.categorical_columns_casing_detection(data)
        return data
    
    def categorical_columns_casing_detection(self,data):
        """Produces a list of text columns that contain casing issues in the text."""
        
        print("Casing Check Please Stop!")
        text_columns=list(data.select_dtypes(exclude=[np.number]).columns)
        global categorical_casing_list
        categorical_casing_list=[]
        data_lower=deepcopy(data)
        for col in text_columns:
            data_lower[col]=data[col].astype(str).str.lower()
            if(len(data[col].unique())==len(data_lower[col].unique())):
                pass
            else:
                categorical_casing_list.append(col)
        print("List which was revised to lower case due to casing issues : "+str(categorical_casing_list))
        data=self.casing_resolve(data,categorical_casing_list)
        return data

    def casing_resolve(self,data,categorical_casing_list):
        """Moves text to lower casing if casing issues are present."""
        
        print("Resolve Casing Issue")
        for col in categorical_casing_list:
            data[col]=data[col].astype(str).str.lower()
        self.imbalanced_data_detection(data,self.target)
        data=self.check_gauss_normalization(data,self.list_numeric)
        return data
    
    def imbalanced_data_detection(self,data,target):
        """Imbalanced Data Detection based on threshold that is difference
        in max count and min count out of all classes is greater than 5 percent"""
        
        print("Checking Dataset For Balance in Data labels")
        col_name='# of occurances'    
        df=(data.groupby(target).size().reset_index(name=col_name))
        res=df[col_name].max()-df[col_name].min();
        if(res>df.shape[0]/20):
            print("Imbalanced Dataset Found for Column : "+str(target))
        else:
            print("Balanced Dataset Found for Column : "+str(target))
    
    def check_gauss_normalization(self,data,numeric_cols):
        """Checks if data is within the conditions for gauss normalization
        then if that is the case, run gauss normalization"""
        
        
        test_df=data[numeric_cols]
        test_df.dropna(inplace=True)
        gauss_columns=[]
        mean = test_df.describe().iloc[1, :]
        std = test_df.describe().iloc[2, :]
        for (col, mean, std) in zip(numeric_cols, mean, std):
            val_df=deepcopy(test_df[test_df[col]>mean-std])
            if((0.66)*test_df[col].shape[0]<(val_df[val_df[col]<mean+std].shape[0]) and (0.70)*test_df[col].shape[0]>(val_df[val_df[col]<mean+std].shape[0])):
                pass
            else:
                gauss_columns.append(col)
        print("Here is the list of Gauss normalized columns : "+str(gauss_columns))
        res=self.perform_gauss_normalization(data,gauss_columns)
    
        return res

    def perform_gauss_normalization(self,data,numeric_cols):
        """Utility Function for check_gauss_normalization"""
        
        print("Performing Power Transformations")
        data.dropna(inplace=True)
        transformed_df=data[numeric_cols]
        pt = PowerTransformer(method='yeo-johnson', standardize=True, copy=True)
        pt.fit(transformed_df)
        transformed_df=pt.transform(transformed_df)
        transformed_df=pd.DataFrame(transformed_df)
        data[numeric_cols]=transformed_df
        return data
        #pt.inverse_transform(transformed_df)

In [40]:
obj=execute_data('zomato.csv','votes')
obj.load_csv()

Reading Data From : ZOMATO.CSV
Removing Whitespace From Data
Removing Redundant Columns From Data
Columns Removed are:['url', 'rate', 'dish_liked']
Dropping Duplicated Data
# of Duplicate Data Records : 40
Extracting Numeric Columns
List of Numeric columns : ['approx_cost(for two people)']
Replace Missing Data
Rows affected are : 28
Identifying categorical variables
['online_order', 'book_table', 'location', 'rest_type', 'listed_in(type)', 'listed_in(city)']
Checking Dataset For Outliers
[['votes', 1001], ['approx_cost(for two people)', 1056]]
Remove Outliers for Dataset
Casing Check Please Stop!
List which was revised to lower case due to casing issues : ['address', 'name']
Resolve Casing Issue
Checking Dataset For Balance in Data labels
Imbalanced Dataset Found for Column : votes
Here is the list of Gauss normalized columns : ['votes', 'approx_cost(for two people)']
Performing Power Transformations


,address,name,online_order,book_table,votes,phone,location,rest_type,cuisines,approx_cost(for two people),...,listed_in(city)_Lavelle Road,listed_in(city)_MG Road,listed_in(city)_Malleshwaram,listed_in(city)_Marathahalli,listed_in(city)_New BEL Road,listed_in(city)_Old Airport Road,listed_in(city)_Rajajinagar,listed_in(city)_Residency Road,listed_in(city)_Sarjapur Road,listed_in(city)_Whitefield
0,"942, 21st main road, 2nd stage, banashankari, ...",jalsa,Yes,Yes,1.496096,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",1.060403,...,0,0,0,0,0,0,0,0,0,0
1,"2nd floor, 80 feet road, near big bazaar, 6th ...",spice elephant,Yes,No,1.504019,080 41714161,Banashankari,Casual Dining,"Chinese, North Indian, Thai",1.060403,...,0,0,0,0,0,0,0,0,0,0
2,"1112, next to kims medical college, 17th cross...",san churro cafe,Yes,No,1.583712,+91 9663487993,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",1.060403,...,0,0,0,0,0,0,0,0,0,0
3,"1st floor, annakuteera, 3rd stage, banashankar...",addhuri udupi bhojana,No,No,0.427712,+91 9620009302,Banashankari,Quick Bites,"South Indian, North Indian",-0.554334,...,0,0,0,0,0,0,0,0,0,0
4,"10, 3rd floor, lakshmi associates, gandhi baza...",grand village,No,No,0.728240,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"North Indian, Rajasthani",0.592738,...,0,0,0,0,0,0,0,0,0,0
5,"37, 5-1, 4th floor, bosco court, gandhi bazaar...",timepass dinner,Yes,No,0.993258,+91 9980040002\r\n+91 9980063005,Basavanagudi,Casual Dining,North Indian,0.592738,...,0,0,0,0,0,0,0,0,0,0
6,"19/1, new timberyard layout, beside satellite ...",rosewood international hotel - bar & restaurant,No,No,-0.601612,+91 9731716688\r\n080 26740366,Mysore Road,Casual Dining,"North Indian, South Indian, Andhra, Chinese",1.060403,...,0,0,0,0,0,0,0,0,0,0
7,"2469, 3rd floor, 24th cross, opposite bda comp...",onesta,Yes,Yes,2.127326,080 48653961\r\n080 48655715,Banashankari,"Casual Dining, Cafe","Pizza, Cafe, Italian",0.592738,...,0,0,0,0,0,0,0,0,0,0
8,"1, 30th main road, 3rd stage, banashankari, ba...",penthouse cafe,Yes,No,1.054972,+91 8884135549\r\n+91 9449449316,Banashankari,Cafe,"Cafe, Italian, Continental",0.843947,...,0,0,0,0,0,0,0,0,0,0
9,"2470, 21 main road, 25th cross, banashankari, ...",smacznego,Yes,No,1.276337,+91 9945230807\r\n+91 9743804471,Banashankari,Cafe,"Cafe, Mexican, Italian, Momos, Beverages",0.450314,...,0,0,0,0,0,0,0,0,0,0
